To create a bipartite graph in Neo4j where users and movies are separated into two groups and the links are given by the ratings, you can use the following Cypher queries:
- Load the CSV files and create nodes for users and movies:

```sql
LOAD CSV WITH HEADERS FROM "file:///C:/movies.csv" AS line
MERGE (m:Movie {movieId: line.movieId, title: line.title})
```

```sql
LOAD CSV WITH HEADERS FROM "file:///C:/ratings.csv" AS line
MERGE (u:User {userId: line.userId})
```

- Create relationships between users and movies based on the ratings:

```sql
LOAD CSV WITH HEADERS FROM "file:///C:/ratings.csv" AS line
MATCH (u:User {userId: line.userId}), (m:Movie {movieId: line.movieId})
MERGE (u)-[:RATED {rating: toFloat(line.rating)}]->(m)
```

These queries will create a bipartite graph with User and Movie nodes and RATED relationships that include the rating given by the user to the movie

To project the bipartite graph onto the two different sets of points (users and movies), you can use the following Cypher queries:

- Project the bipartite graph onto the user nodes:

```sql
MATCH (u1:User)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(u2:User)
WHERE u1 <> u2
MERGE (u1)-[s:SIMILAR]->(u2)
ON CREATE SET s.commonMovies = 1
ON MATCH SET s.commonMovies = s.commonMovies + 1
```
Project the bipartite graph onto the movie nodes:

```sql
MATCH (m1:Movie)<-[r1:RATED]-(u:User)-[r2:RATED]->(m2:Movie)
WHERE m1 <> m2
MERGE (m1)-[s:SIMILAR]->(m2)
ON CREATE SET s.commonUsers = 1
ON MATCH SET s.commonUsers = s.commonUsers + 1
```

These queries will create SIMILAR relationships between users and between movies based on the common movies they have rated and the common users who have rated them, respectively.

In [1]:
# Setup Connection
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"
password = "testtest"
driver = GraphDatabase.driver(uri, auth=(username, password))

in Cypher

```sql
CREATE (n:Person {name: "Alice"})
```

creates a node ( ) with label _Person_ with the property with key _name_ of value _Alice_

In [2]:
def add_node(tx, name):
    tx.run("CREATE (n:Person {name: $name})", name=name)

with driver.session() as session:
    session.execute_write(add_node, "Alice")

in Cypher

```sql
MATCH (n:Person {name: "Alice"}) RETURN n.name AS name
```

In [ ]:
def get_person(tx, name):
    result = tx.run("MATCH (n:Person {name: $name}) RETURN n.name AS name", name=name)
    return [record["name"] for record in result]

with driver.session() as session:
    names = session.execute_read(get_person, "Alice")
    print(names)

in Cypher

```sql
MATCH ()-[r]->() DELETE r
MATCH (n) DELETE n
```

In [4]:
def delete_all(tx):
    tx.run("MATCH ()-[r]->() DELETE r")
    tx.run("MATCH (n) DELETE n")
        
with driver.session() as session:
    session.execute_write(delete_all)

In [6]:
driver.close()

Le nostre query

```sql
MATCH (n:User {userId: 1}) RETURN n.userId AS name
MATCH (m:Movie  {movieId: 1}) RETURN m.title AS name
```